In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor


llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [2]:
@tool
def get_protein_intake(weight: int) -> int:
    """Returns the recommended protein intake for a person based on their weight."""
    return weight * 0.8


In [3]:
tools = [get_protein_intake]

In [4]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [5]:
llm_with_tools = llm.bind_tools(tools)

In [6]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [7]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
list(agent_executor.stream({"input": "I am 80kg, how much protein should I eat?"}))



> Entering new AgentExecutor chain...

Invoking: `get_protein_intake` with `{'weight': 80}`


64.0Based on your weight of 80 kg, you should aim to consume approximately 64 grams of protein per day.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_protein_intake', tool_input={'weight': 80}, log="\nInvoking: `get_protein_intake` with `{'weight': 80}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_UbShrztDIlkzh9W7si5uf8hV', 'function': {'arguments': '{"weight":80}', 'name': 'get_protein_intake'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d576307f90'}, id='run-98418cdd-ad4a-4418-a4a6-78c2ce881b87', tool_calls=[{'name': 'get_protein_intake', 'args': {'weight': 80}, 'id': 'call_UbShrztDIlkzh9W7si5uf8hV'}], tool_call_chunks=[{'name': 'get_protein_intake', 'args': '{"weight":80}', 'id': 'call_UbShrztDIlkzh9W7si5uf8hV', 'index': 0}])], tool_call_id='call_UbShrztDIlkzh9W7si5uf8hV')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_UbShrztDIlkzh9W7si5uf8hV', 'function': {'arguments': '{"weight":80}